# SANSANのコンタクトのCSVをSQLServerに登録する

In [ ]:
!pip install pyodbc
!pip install sqlalchemy==1.4.46
!pip install pandas

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# ユーザ(SANSAN_USER)のTRUNCATE＋登録

In [ ]:
dtype = {
    'ユーザID': 'Int64',
    # 'ＦＩＬ': 'Int64',
    '削除フラグ': 'Int64',
}

csv_file = r"D:\Users\python\sansan\notebooks\data\sansan_user.csv"
user_df = pd.read_csv(csv_file, dtype=dtype)
column_mappings = {
    '所属部署 *': 'dpt',
    'ユーザID *': 'id',
    'ユーザ名 *': 'name',
    # 'ユーザ名の読み': 'NAME_READABLE',
    'メールアドレス *': 'mail',
    # 'サブメールアドレス': 'SUB_MAIL',
    # '利用開始日 *': 'START_DATE',
    # '言語 (システム上の表示言語)': 'LANG',
    # 'メール受信形式': 'MAILER',
    # 'SAML Name ID': 'SAML_NAME_ID',
    # 'ユーザ区分': 'CATERGORY',
    # '[権限] 全データ更新': 'AUTH_UPDATE',
    # '[権限] 名刺・コンタクトのダウンロード': 'AUTH_DL_CARD',
    # '[権限] メール配信': 'AUTH_MAIL',
    # '[権限] 案件管理': 'AUTH_ANKEN',
    # '[権限] API・Zapier連携': 'AUTH_API',
    # '[権限] 利用実績の確認': 'AUTH_RECORD',
    # '[権限] 会社・組織ツリーのダウンロード': 'AUTH_DL_CO',
    # 'アカウント状況': 'STATUS',
    '削除フラグ': 'del_flg',
}

req_columns = list(column_mappings.keys())
fltr_user_df = user_df[req_columns]
fltr_user_df = fltr_user_df.rename(columns=column_mappings)

# 部署に'産業資材事業部'を含む行のみを選択
pattern = '産業資材事業部'
fltr_user_df = fltr_user_df[fltr_user_df['dpt'].str.contains(pattern, regex=True)]

# 最初のスペースまでの文字列を抽出
fltr_user_df['branch'] = fltr_user_df['dpt'].str.split().str[0]

fltr_user_df['del_flg'].fillna(0, inplace=True)

print(fltr_user_df)


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# SQL Serverへの接続設定
server = 'SO-PC'
database = 'datamart'
username = 'gpkadmin'
password = '19vK8xEQ'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server'

# SQLAlchemyエンジンを作成
engine = create_engine(connection_string)

In [ ]:
# Truncateするテーブル名
user_table_name = "sansan_user"

# テーブルをtruncateする
with engine.begin() as conn:
    conn.execute(text(f"TRUNCATE TABLE {user_table_name}"))


In [ ]:
# SQL Serverにデータをインポート
fltr_user_df.to_sql(user_table_name, con=engine, if_exists='append', index=False)

# コンタクト登録

In [ ]:
csv_file = r"D:\Users\python\sansan\notebooks\data\contactlist_ 20230321_20231126_utf8.csv"
contact_df = pd.read_csv(csv_file)
contact_df = contact_df.drop(['社外出席者 [部署 役職]'], axis=1)
column_mappings = {
    'コンタクト日': 'contact_date',
    '時間': 'contact_time',
    '区分': 'catergory',
    '重要フラグ': 'importance',
    '訪問目的': 'purpose',
    '訪問先': 'destination',
    '商談アイテム': 'product',
    '社外出席者': 'customer',
    '部署': 'customer_dept',
    '役職': 'customor_pos',
    '社内出席者': 'salesperson_visited',
    'メモ': 'contact_details',
    '登録者': 'salesperson_registered',
}
req_columns = list(column_mappings.keys())
fltr_contact_df = contact_df[req_columns]

fltr_contact_df.rename(columns=column_mappings, inplace=True)

fltr_contact_df['importance'] = fltr_contact_df['importance'].fillna('')

# 最初のスペースまでの文字列を抽出
fltr_contact_df['customer_company'] = fltr_contact_df['customer'].str.split().str[0]

# 2つ目のスペースまでの文字列を抽出
fltr_contact_df['customer_name'] = fltr_contact_df['customer'].apply(lambda x: ' '.join(x.split()[1:3]))


# ユーザを名前で結合
user_brn_df = fltr_user_df[list(['name', 'branch'])]
merged_df = fltr_contact_df.merge(user_brn_df, left_on='salesperson_registered', right_on='name' , how='inner').drop(columns=['name'])

print(merged_df)

In [ ]:
# SQL Serverへの接続設定
server = 'SO-PC'
database = 'datamart'
username = 'gpkadmin'
password = '19vK8xEQ'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server'

# SQLAlchemyエンジンを作成
engine = create_engine(connection_string)

# SQL Serverにデータをインポート
# 1万件で13分かかる
merged_df.to_sql('sansan_contact', con=engine, if_exists='append', index=False)